In [52]:
import os
import re
from collections import defaultdict

path = '/home/naztom/tvd/mp3_sorter_tagger/data/'
os.chdir(path)

def album_has_year(dirname):
    return dirname[:4].isnumeric() or dirname[1:5].isnumeric()

# TODO: check if year string or int later on
def split_album_year_name(dirname):
    if not album_has_year(dirname):
        year, name = None, dirname
    else:
        try:
            year = int(dirname[:4])
            name = dirname[4:].strip(' -_]')
        except Exception:
            year = int(dirname[1:5])
            name = dirname[5:].strip(' -_]')
    return year, name

def split_album_name_type(dirname):
    types = '|'.join(['ep', 'single', 'remixes'])
    pattern = fr'([\(\[^\w]?({types})[\)\]^\w]?)'
    pattern_compiled = re.compile(pattern, re.I)
    res = re.search(pattern_compiled, dirname)
    if res:
        type = res.group(2)
        name = dirname.split(res.group(1))[0].strip()
    else:
        type, name = None, dirname
    name_original = name.rstrip(' -')
    name_title = name_original.title()
    changed = name_original != name_title
    return name_title, type, name_original, changed

def check_title(old, new):
    for idx, (letter_old, letter_new) in enumerate(zip(old, new)):
        prev_old = old[idx-1]
        if letter_old != letter_new and prev_old != ' ' and idx != 0:
            return False
    return True

to_rename = defaultdict(dict)
all_good = defaultdict(dict)
incorrect_title = defaultdict(dict)

for band in os.listdir():
    os.chdir(path + band)
    for folder in os.listdir():
        year, name = split_album_year_name(folder)
        name, type, name_original, name_changed = split_album_name_type(name)
        full = ' - '.join([str(year), name]) if year else name
        full += f' [{type}]' if type else ''
        if name_changed and not check_title(name, name_original):
            incorrect_title[band][folder] = full
        elif folder != full:
            to_rename[band][folder] = full
        else:
            all_good[band][folder] = full

display(incorrect_title)
display(to_rename)
display(all_good)



defaultdict(dict,
            {'Accessory': {'2001 - Jukka2147.DE': '2001 - Jukka2147.De'},
             'Aesthetic Perfection': {"2011 the devil's in the details EP": "2011 - The Devil'S In The Details [EP]"}})

defaultdict(dict,
            {'Accessory': {'2011 - Underbeat EP': '2011 - Underbeat [EP]',
              '[2007] - Holy Machine (EP)': '2007 - Holy Machine [EP]',
              'More than Machinery': 'More Than Machinery',
              '2019 - No Man Covers - Single': '2019 - No Man Covers [Single]'},
             'Aesthetic Perfection': {'2012 a nice place to destroy EP': '2012 - A Nice Place To Destroy [EP]',
              '2011 all beauty destroyed - Remixes': '2011 - All Beauty Destroyed [Remixes]',
              '2015 Blood Spills Not Far From The Wound': '2015 - Blood Spills Not Far From The Wound'}})

defaultdict(dict,
            {'Accessory': {'2023 - Jupiter': '2023 - Jupiter',
              '2005 - Forever And Beyond': '2005 - Forever And Beyond',
              '1997 - Electronic Controlled Mind [EP]': '1997 - Electronic Controlled Mind [EP]'},
             '18 Summers': {'2002 - Virgin Mary': '2002 - Virgin Mary'}})

In [51]:
old = '[2005] - Forever and Beyond'
new = '2005 - Forever And Beyond'
band = 'Accessory'
os.chdir(path + band)
print(os.listdir())
os.rename(old, new)
print(os.listdir())


['[2005] - Forever and Beyond', '2011 - Underbeat EP', '[2007] - Holy Machine (EP)', '2023 - Jupiter', 'More than Machinery', '1997 - Electronic Controlled Mind [EP]', '2019 - No Man Covers - Single', '2001 - Jukka2147.DE']
['2011 - Underbeat EP', '[2007] - Holy Machine (EP)', '2023 - Jupiter', 'More than Machinery', '2005 - Forever And Beyond', '1997 - Electronic Controlled Mind [EP]', '2019 - No Man Covers - Single', '2001 - Jukka2147.DE']
